In [1]:
import json

In [2]:
with open("../mock_dogs_db/db_combined_n177.json", "r") as json_file:
    DOGS = json.load(json_file)

In [3]:
def interpretation(value):

    translation = {word:no for word, no in zip(["zero", "one", "two", "three", "four", "five"], [0,1,2,3,4,5])}

    value = translation[value]

    if value >= 3:
        interpret = list(range(value, 6))
    if value <= 2:
        interpret = list(range(0, value+1))
    return interpret

In [4]:
def filter_dogs(scores, db=DOGS):

    scores = [(feat, score) for feat, score in scores if score != None]

    if scores == []:
        return set([dog["name"] for dog in db])
    else:
        hits=set()
        for i, (feature, score) in enumerate(scores):
            score_meaning = interpretation(score)
            if i == 0:
                hits.update([dog["name"] for dog in db if dog[feature] in score_meaning])

            else:
                new_dogs = [dog["name"] for dog in db if dog[feature] in score_meaning]
                hits = hits.intersection(new_dogs)
        
        return hits

In [5]:
def filler(score):
    global scoreboard
    
    for i, (feat, value) in enumerate(scoreboard):
        if value == None:
            # does feat=value not exist for any dog, then break
            snoop_dog = [dog["name"] for dog in DOGS if int(dog[feat]) in interpretation(score) and dog["name"] in remaining_dogs]
            if len(snoop_dog) > 0:
                scoreboard[i] = (feat, score)
                break
            else:
                print("NO DOG MATCH!")
                break

In [6]:
def go_lden_retriever(dog2feature="name", name=None, feature2dog=None, multiple_conditions=None, database="local"):

    if database == "local":
        if multiple_conditions != None: # gets the dog names (list) of dogs having a value on param feature2dog that is in param multiple_conditions 
            dogs_in_the_yard = [dog["name"] for dog in DOGS if dog[feature2dog] in multiple_conditions] 
        if dog2feature == None: # gets the dict/json for the dog named by param name (single element of list) 
            dogs_in_the_yard = [dog for dog in DOGS if dog["name"] == name] 
        if name == None and feature2dog==None: # gets all dog names in the database; NOT POSSIBLE FOR API
            dogs_in_the_yard = [dog["name"] for dog in DOGS]
        if dog2feature != None and name != None: # gets the score for a feature defined by param dog2feature of a dog named by param name
            dogs_in_the_yard = [dog[dog2feature] for dog in DOGS if dog["name"] == name]

    return dogs_in_the_yard

In [7]:
def json2content(json):

    tr_scale = ["very hard to", "hard to", "quite hard", "easy", "quite easy", "very easy"]
    sh_ba_scale = ["almost nothing", "very litte", "little", "to some extent", "much", "very much"]
    en_scale = ["very low", "low", "quite low", "quite high", "high", "very high"]
    pr_scale = ["very much not", "quite not", "somewhat not", "somewhat", "quite", "very"]
    go_scale = ["extremly bad", "very bad", "bad", "good", "very good", "extremly good"]

    content = f"{json['name']} is a dog which is {tr_scale[int(json['trainability'])]} to train. It sheds {sh_ba_scale[int(json['shedding'])]}. It is {en_scale[int(json['energy'])]} in energy and barks {sh_ba_scale[int(json['barking'])]}. It is {pr_scale[int(json['protectiveness'])]} protective. It is {go_scale[int(json['good_with_children'])]} with children"

    return content

In [8]:
def dog_describer(my_dog):
    data_entry = go_lden_retriever(dog2feature=None, name=my_dog, feature2dog="name")[0]
    content = json2content(data_entry)
    return content

In [9]:
def get_comparison(target, compare_with, feature):

    trg_value = int(go_lden_retriever(dog2feature=feature, name=target, feature2dog="name")[0])
    lm_value  = int(go_lden_retriever(dog2feature=feature, name=compare_with, feature2dog="name")[0])

    if trg_value-lm_value == 0:
        evaluation = "zero"
    else:
        if trg_value>lm_value:
            evaluation = "pos"
        else:
            evaluation = "neg"

    semantics = {
    "trainability": {
        "pos": "is easier to train than",
        "neg": "is harder to train than",
        "zero": "is as easy to train as",
        }, 
    "shedding": {
        "pos": "sheds more than",
        "neg": "sheds less than",
        "zero": "sheds as much as",
        },  
    "energy": {
        "pos": "has more energy than",
        "neg": "has less energy than",
        "zero": "has as much energy as",
        }, 
    "barking": {
        "pos": "barks more than",
        "neg": "barks less than",
        "zero": "barks as much as",
        },  
    "protectiveness": {
        "pos": "is more protective than",
        "neg": "is less protective than",
        "zero": "is as protective as",
        },  
    "good_with_children": {
        "pos": "is better with children than",
        "neg": "is worse with children than",
        "zero": "is as good with children as",
        }
    }

    comparison = f"{target} {semantics[feature][evaluation]} {compare_with}"

    return comparison

In [10]:
def dog_comparator(target, compare_with, feature):
    compare_str = get_comparison(target, compare_with, feature)
    return compare_str

In [11]:
def get_top(feature, mode, R_set):
    R_set = list(R_set)

    first_dog = R_set[0]
    top_dog = set()
    top_dog.add(first_dog)

    top_value = [int(x) for x in go_lden_retriever(dog2feature=feature, name=first_dog, feature2dog="name")][0]
    for f_dog in R_set[1:]:
        dog_json = go_lden_retriever(dog2feature=None, name=f_dog, feature2dog="name")[0]
        value = int(dog_json[feature])
        if value == top_value:
            top_dog.add(dog_json["name"])

        if mode == "most":
            if value > top_value:
                top_dog = set()
                top_dog.add(dog_json["name"])
                top_value = value
        if mode == "least":
            if value < top_value:
                top_dog = set()
                top_dog.add(dog_json["name"])
                top_value = value                              

    top_dog = list(top_dog)
    return top_dog

In [12]:
def say_top(top, feature, pol):

    template = {
    "trainability": "The ___ trainable dog* & ", 
    "shedding": "The dog* that shed? the ___ & ",
    "energy": "The dog* with ___ energy & ",
    "barking": "The dog* that bark? the ___ & ",  
    "protectiveness": "The ___ protective dog* & ",  
    "good_with_children": "The dog* that & ___ good with children & "
    }

    sent = template[feature]
    if len(top) > 1: #plural
        sent = sent.replace("*", "s")
        sent = sent.replace("&", "are")
        sent = sent.replace("?", "")
    else:
        sent = sent.replace("*", "")
        sent = sent.replace("&", "is")
        sent = sent.replace("?", "s")

    sent = sent.replace("___", pol)

    sent = sent + ", ".join(top)

    return sent

In [13]:
def most_dog_finder(feature):
    most = get_top(feature, "most", remaining_dogs) # a list
    most_str = say_top(most, feature, "most")   
    return most_str


def least_dog_finder(feature):
    least = get_top(feature, "least", remaining_dogs)
    least_str = say_top(least, feature, "least") 
    return least_str

In [14]:
def f(score):
    global remaining_dogs
    print("Turns left", 4-len(["x" for feat, value in scoreboard if value != None]))
    filler(score)
    print("Scoreboard:", scoreboard)
    remaining_dogs = filter_dogs(scoreboard)
    print("No of dogs:", len(remaining_dogs))
    print("Dogs:", ", ".join(list(remaining_dogs)[:10]))

In [17]:
scoreboard = [
    ("trainability", None), 
    ("shedding", None), 
    ("energy", None), 
    ("barking", None), 
    ("protectiveness", None)]
remaining_dogs = filter_dogs(scoreboard)

In [23]:
f("two")

Turns left 0
NO DOG MATCH!
Scoreboard: [('trainability', 'three'), ('shedding', 'one'), ('energy', 'three'), ('barking', 'one'), ('protectiveness', None)]
No of dogs: 3
Dogs: Bergamasco Sheepdog, Irish Water Spaniel, Briard


In [24]:
dog_describer("Briard")

'Briard is a dog which is easy to train. It sheds very litte. It is quite high in energy and barks very litte. It is quite protective. It is good with children'

In [ ]:
dog_comparator("Porcelaine", "Golden Retriever", "barking")

In [ ]:
most_dog_finder("energy")

In [ ]:
least_dog_finder("barking")